In [27]:
import pandas as pd

import torchmetrics
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import CrossEntropyLoss
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split

### Training a Neural Network:

1. ~~Create a model~~
2. Choose a loss function
3. Create a dataset
4. Define an optimizer
5. Run a training loop, where for each sample of the dataset, we repeat: (akin to .fit in sklearn)
    - Calculating loss (forward pass)
    - Calculating local gradients (backward pass)
    - Updating model parameters

In [26]:
### DATA PROCESSING

# np_array = np.array(array)
# np_tensor = torch.from_numpy(np_array)

data_pd = pd.read_csv('~/Documents/Data Science/programs/iris_neural_net/iris_data.csv')

x = data_pd.drop(['species'], axis=1)
y = data_pd.species

encoder = LabelEncoder()
y = encoder.fit_transform(y)

scaler = StandardScaler()
x = scaler.fit_transform(x)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, random_state = 888)

features = torch.from_numpy(x_train)
labels = F.one_hot(torch.from_numpy(y_train))

# print(labels)

In [22]:
### FORWARD PASS

dataset = TensorDataset(features.float(), labels.float())
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)
num_epochs = 10

model = nn.Sequential(
    nn.Linear(4, 6),
    nn.Linear(6, 3),
    nn.Softmax(dim=-1)
)

scores = model(input)
# _, preds = torch.max(output, dim=1)
# preds = F.one_hot(preds, num_classes=3)

# CrossEntropyLoss for classification, MSELoss for regression
criterion = CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

for epoch in range(num_epochs):
    for data in dataloader:
        optimizer.zero_grad()
        feature, target = data
        pred = model(feature)
        loss = criterion(pred, target)
        loss.backward()
        optimizer.step()

print(model[1].bias.grad)

tensor([-0.0274,  0.0054,  0.0220])
